
<img src="../../img/ods_stickers.jpg">

## <center> [mlcourse.ai](https://mlcourse.ai) – открытый курс OpenDataScience по машинному обучению 
    
Автор материала: Юрий Кашницкий (@yorko в Slack ODS). Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [18]:
# отключим всякие предупреждения Anaconda
import warnings

warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics.regression import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [19]:
data = pd.read_csv('data/winequality-white.csv', sep=';')

In [20]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [23]:
independent_columns_names = [x for x in data if x != 'quality']
y = data['quality']
X = data[independent_columns_names]

train_size = int(0.7 * X.shape[0])

X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, y, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [24]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [25]:
y_holdout_pred = linreg.predict(X_holdout_scaled)
y_train_pred = linreg.predict(X_train_scaled)
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_train_pred))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, y_holdout_pred))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [26]:
feature_importances = linreg.coef_
linreg_coef = pd.DataFrame({"feat_importances":np.abs(feature_importances), "features":independent_columns_names})
linreg_coef.sort_values(by="feat_importances", ascending=False)

,feat_importances,features
7,0.665720,density
3,0.538164,residual sugar
1,0.192260,volatile acidity
8,0.150036,pH
10,0.129533,alcohol
0,0.097822,fixed acidity
9,0.062053,sulphates
5,0.042180,free sulfur dioxide
6,0.014304,total sulfur dioxide
4,0.008127,chlorides


Residual Sugar

## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [30]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [31]:
lasso1_coef = pd.DataFrame({"feat_importances":np.abs(lasso1.coef_), "features":independent_columns_names})
lasso1_coef.sort_values(by="feat_importances", ascending=False)

,feat_importances,features
10,0.322425,alcohol
3,0.256363,residual sugar
7,0.235492,density
1,0.188479,volatile acidity
8,0.067277,pH
5,0.043088,free sulfur dioxide
9,0.029722,sulphates
4,0.002747,chlorides
0,0.000000,fixed acidity
2,0.000000,citric acid


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [32]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [33]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [35]:
lasso_cv_coef = pd.DataFrame({"feat_importances":np.abs(lasso_cv.coef_), "features":independent_columns_names})
lasso_cv_coef.sort_values(by="feat_importances", ascending=False)

,feat_importances,features
7,0.648161,density
3,0.526883,residual sugar
1,0.192049,volatile acidity
8,0.146549,pH
10,0.137115,alcohol
0,0.093295,fixed acidity
9,0.060939,sulphates
5,0.042698,free sulfur dioxide
6,0.012969,total sulfur dioxide
4,0.006933,chlorides


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [37]:
y_holdout_pred_lcv = lasso_cv.predict(X_holdout_scaled)
y_train_pred_lcv = lasso_cv.predict(X_train_scaled)
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_train_pred_lcv))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, y_holdout_pred_lcv))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [45]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [46]:
y_holdout_pred_rfr = forest.predict(X_holdout_scaled)
y_train_pred_rfr = forest.predict(X_train_scaled)
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, y_train_pred_rfr))
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train, 
                                                                       scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, y_holdout_pred_rfr))

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [47]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(estimator=forest, param_grid=forest_params)
locally_best_forest.fit(X_train_scaled, y_train)

GridSearchCV(estimator=RandomForestRegressor(random_state=17),
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]})

In [48]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 0.4939384585901731)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [49]:
locally_best_forest.best_estimator_
best_forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
best_forest.fit(X_train, y_train)

RandomForestRegressor(max_depth=19, max_features=7, random_state=17)

In [50]:
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(locally_best_forest.
                                                        best_estimator_,
                                                        X_train_scaled, y_train, 
                                                        scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, 
                                                             locally_best_forest.predict(
                                                                 X_holdout_scaled)))

Mean squared error (cv): 0.398
Mean squared error (test): 0.366


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [51]:
rf_importance = pd.DataFrame({"feat_importances":best_forest.feature_importances_, "features":independent_columns_names})
rf_importance.feat_importances = rf_importance.feat_importances.astype(float)
rf_importance.sort_values(by="feat_importances", ascending=False)

,feat_importances,features
10,0.214825,alcohol
1,0.117627,volatile acidity
5,0.113509,free sulfur dioxide
7,0.086530,density
8,0.072646,pH
3,0.072529,residual sugar
6,0.071644,total sulfur dioxide
4,0.067819,chlorides
0,0.061873,fixed acidity
2,0.061500,citric acid


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**